# Estimation of gamma,Ks inclusive cross section from Radphi data

In [1]:
from pyxrootd import client as xclient
from gluex.jupyroot.treeview import treeview
import ROOT
%jsroot on
import numpy as np
import os
os.chdir("/srv/jupyter/RadPhi studies")
#os.remove("radphi-kshorts.root")

In [3]:
xrdurl = "root://nod28.phys.uconn.edu/"
xrdpath = "/Gluex/radphi/pass-9-2020/"
xfs = xclient.FileSystem(xrdurl)
status, listing = xfs.dirlist(xrdpath)
tchain = ROOT.TChain("h1","")
for entry in listing['dirlist']:
    if ".root" in entry['name']:
        tchain.Add(xrdurl + xrdpath + entry['name'])
        if len(tchain.GetListOfFiles()) == 1:
            tchain.Print()
print("tchain has nfiles=", len(tchain.GetListOfFiles()))
#print("tchain has nentries=", tchain.GetEntries())

tchain has nfiles= 2578
******************************************************************************
*Chain   :h1        : root://nod28.phys.uconn.edu//Gluex/radphi/pass-9-2020/r7773-0.root *
******************************************************************************
******************************************************************************
*Tree    :h1        : Radphi                                                 *
*Entries :   420000 : Total =      2101775546 bytes  File  Size =  867370797 *
*        :          : Tree compression factor =   2.42                       *
******************************************************************************
*Br    0 :runno     : runno/I                                                *
*Entries :   420000 : Total  Size=    1682915 bytes  File Size  =      11245 *
*Baskets :       29 : Basket Size=     183808 bytes  Compression= 149.58     *
*............................................................................*
*Br    1 :eventn

In [4]:
tview = treeview(tchain, "radphi-kshorts.root")
tview.enable_dask_cluster(dclient)

## Survey of photon multiplicity

In [5]:
ROOT.EnableImplicitMT()
rdf = ROOT.RDataFrame(tchain)
for cname in ("nphot", "nfrwd"):
    h = rdf.Histo1D((f"h{cname}", cname, 11, -0.5, 10.5), cname)
    tview.register_histogram(f"h{cname}", h)

In [6]:
n = tview.draw(['', ''])
panel = 1
for hname in ("hnphot", "hnfrwd"):
    h = tview.evaluate(hname)
    h.GetXaxis().SetTitle("reconstructed photon count")
    h.GetYaxis().SetTitle("events")
    tview.current_canvas.cd(panel)
    h.Draw()
    panel += 1

In [7]:
rdf4f = rdf.Filter("nfrwd==4")
rdf4f_state = rdf4f.Define("state4f", """
    struct state4f {
        double mass4g;
    } state;
    double p[4] = {0,0,0,0};
    for (int i=0; i<nfrwd; i++) {
        for (int mu=0; mu<4; ++mu) {
            p[mu] += pvect[i][mu];
        }
    }
    state.mass4g = sqrt(p[3]*p[3] - p]0]*p[0] - p[1]*p[1] - p[2]*p[2]);
    return state;
""")
rdf4f_mass4g = rdf4f_state.Define("mass4g", "state4f.massg4")
rdf4f_hmass4g = rdf4f_mass4g.Histo1D(("hmass4g", "4g mass distribution, nfrwd==4", 1000, 0, 1.2), "mass4g") 
h = tview.register_histogram("hmass4g", rdf4f_hmass4g)
%time tview.evaluate("hmass4g")

TypeError: Template method resolution failed:
  ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Define(string_view name, string_view expression) =>
    runtime_error: TTree leaf pvect.pvect has both a leaf count and a static length. This is not supported.
  Failed to instantiate "Define(ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>&,std::string,std::string)"
  Failed to instantiate "Define(ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>*,std::string,std::string)"
  Failed to instantiate "Define(ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>,std::string,std::string)"